In [159]:
# Python 2 & 3 Compatibility
from __future__ import print_function, division

# Necessary imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import seaborn as sns
from seaborn import plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV

%matplotlib inline

In [160]:
from dbfread import DBF

In [161]:
# This file is too large to track in this Github repo

table = DBF('../data/raw/hennepin_county_metrogis_parcels/hennepin_county_metrogis_parcels.dbf', load=True, raw=True)

In [195]:
cols = table.field_names
cols

['COUNTY_ID',
 'PIN',
 'BLDG_NUM',
 'PREFIX_DIR',
 'PREFIXTYPE',
 'STREETNAME',
 'STREETTYPE',
 'SUFFIX_DIR',
 'UNIT_INFO',
 'CITY',
 'CITY_USPS',
 'ZIP',
 'ZIP4',
 'PLAT_NAME',
 'BLOCK',
 'LOT',
 'ACRES_POLY',
 'ACRES_DEED',
 'USE1_DESC',
 'USE2_DESC',
 'USE3_DESC',
 'USE4_DESC',
 'MULTI_USES',
 'LANDMARK',
 'OWNER_NAME',
 'OWNER_MORE',
 'OWN_ADD_L1',
 'OWN_ADD_L2',
 'OWN_ADD_L3',
 'TAX_NAME',
 'TAX_ADD_L1',
 'TAX_ADD_L2',
 'TAX_ADD_L3',
 'HOMESTEAD',
 'EMV_LAND',
 'EMV_BLDG',
 'EMV_TOTAL',
 'TAX_CAPAC',
 'TOTAL_TAX',
 'SPEC_ASSES',
 'TAX_EXEMPT',
 'XUSE1_DESC',
 'XUSE2_DESC',
 'XUSE3_DESC',
 'XUSE4_DESC',
 'DWELL_TYPE',
 'HOME_STYLE',
 'FIN_SQ_FT',
 'GARAGE',
 'GARAGESQFT',
 'BASEMENT',
 'HEATING',
 'COOLING',
 'YEAR_BUILT',
 'NUM_UNITS',
 'SALE_DATE',
 'SALE_VALUE',
 'SCHOOL_DST',
 'WSHD_DIST',
 'GREEN_ACRE',
 'OPEN_SPACE',
 'AG_PRESERV',
 'AGPRE_ENRD',
 'AGPRE_EXPD',
 'PARC_CODE',
 'Shape_area',
 'Shape_len']

In [199]:
# create a dataframe

df = pd.DataFrame(table.records)

In [200]:
df.sample(5)

,COUNTY_ID,PIN,BLDG_NUM,PREFIX_DIR,PREFIXTYPE,STREETNAME,STREETTYPE,SUFFIX_DIR,UNIT_INFO,CITY,...,SCHOOL_DST,WSHD_DIST,GREEN_ACRE,OPEN_SPACE,AG_PRESERV,AGPRE_ENRD,AGPRE_EXPD,PARC_CODE,Shape_area,Shape_len
18632,b'053',b'053-3402924240224',b'2500 ',b' ',b' ',b'BLAISDELL AVE S ',b' ',b' ',b'202 ',b'MINNEAPOLIS ',...,b'001 ',b'Middle Mississippi ...,b'N',b'N',b'N',b' ',b' ',b' 0',b' 3.00064830874e+003',b' 2.19955517527e+002'
104900,b'053',b'053-3411922330070',b'6268 ',b' ',b' ',b'EMPIRE LA N ',b' ',b' ',b' ',b'MAPLE GROVE ',...,b'279 ',b'Lower Minnesota River ...,b'N',b'N',b'N',b' ',b' ',b' 0',b' 1.09068264126e+003',b' 1.33445617621e+002'
373980,b'053',b'053-2711823330014',b'1003 ',b' ',b' ',b'COX FARM RD ',b' ',b' ',b' ',b'ORONO ',...,b'278 ',b'Minnehaha Creek ...,b'N',b'N',b'N',b' ',b' ',b' 0',b' 8.10613777143e+003',b' 3.64894065657e+002'
421483,b'053',b'053-1102924440143',b'1828 ',b' ',b' ',b'QUINCY ST N E ',b' ',b' ',b' ',b'MINNEAPOLIS ',...,b'001 ',b'Middle Mississippi ...,b'N',b'N',b'N',b' ',b' ',b' 0',b' 4.31402800650e+002',b' 9.20192597310e+001'
378237,b'053',b'053-0302824420139',b'3500 ',b' ',b' ',b'STEVENS AVE S ',b' ',b' ',b' ',b'MINNEAPOLIS ',...,b'001 ',b'Middle Mississippi ...,b'N',b'N',b'N',b' ',b' ',b' 0',b' 3.76159905025e+002',b' 8.10102390853e+001'


In [201]:
df.shape

(429570, 67)

In [202]:
# Turn the table from binary to a utf-8 encoded string

def decode_binary(df, cols):
    decode = lambda x : x.decode('utf-8')
    for col in cols:
        df[col] = df[col].apply(decode)
        

In [203]:
decode_binary(df, cols)
df.head()

,COUNTY_ID,PIN,BLDG_NUM,PREFIX_DIR,PREFIXTYPE,STREETNAME,STREETTYPE,SUFFIX_DIR,UNIT_INFO,CITY,...,SCHOOL_DST,WSHD_DIST,GREEN_ACRE,OPEN_SPACE,AG_PRESERV,AGPRE_ENRD,AGPRE_EXPD,PARC_CODE,Shape_area,Shape_len
0,053,053-0911821410021,54,,,ADDRESS UNASSIGNED,,,,CRYSTAL,...,281,Shingle Creek ...,N,N,N,,,0,7.25247600000e-002,9.55874206426e-001
1,053,053-1311821120005,4400 1/2,,,LYNDALE AVE N,,,,MINNEAPOLIS,...,001,Shingle Creek ...,N,N,N,,,0,7.25380050000e-002,9.55961240593e-001
2,053,053-1311724440113,2325,,,MONTCLAIR LA,,,,MOUND,...,,...,N,N,N,,,0,7.25352550000e-002,9.55943566274e-001
3,053,053-2911821440091,28,,,ADDRESS UNASSIGNED,,,,GOLDEN VALLEY,...,281,Bassett Creek ...,N,N,N,,,0,7.25358400000e-002,9.55947921559e-001
4,053,053-0111823440003,80,,,ADDRESS UNASSIGNED,,,,MEDINA,...,284,...,N,N,N,,,0,7.25419200000e-002,9.55987121801e-001


In [204]:
# Remove white space from around the entries

def strip_entries(df, cols):
    strip = lambda x : x.strip()
    for col in cols:
        df[col] = df[col].apply(strip)

In [205]:
# Remove the county code and dash from the property IDs

def remove_dash(df, cols):
    x_dash = lambda x : x.split('-')[1] if '-' in x else x
    for col in cols:
        df[col] = df[col].apply(x_dash)

In [206]:
strip_entries(df, cols)

In [207]:
remove_dash(df, cols)

In [208]:
df.head()

,COUNTY_ID,PIN,BLDG_NUM,PREFIX_DIR,PREFIXTYPE,STREETNAME,STREETTYPE,SUFFIX_DIR,UNIT_INFO,CITY,...,SCHOOL_DST,WSHD_DIST,GREEN_ACRE,OPEN_SPACE,AG_PRESERV,AGPRE_ENRD,AGPRE_EXPD,PARC_CODE,Shape_area,Shape_len
0,053,0911821410021,54,,,ADDRESS UNASSIGNED,,,,CRYSTAL,...,281,Shingle Creek,N,N,N,,,0,002,001
1,053,1311821120005,4400 1/2,,,LYNDALE AVE N,,,,MINNEAPOLIS,...,001,Shingle Creek,N,N,N,,,0,002,001
2,053,1311724440113,2325,,,MONTCLAIR LA,,,,MOUND,...,,,N,N,N,,,0,002,001
3,053,2911821440091,28,,,ADDRESS UNASSIGNED,,,,GOLDEN VALLEY,...,281,Bassett Creek,N,N,N,,,0,002,001
4,053,0111823440003,80,,,ADDRESS UNASSIGNED,,,,MEDINA,...,284,,N,N,N,,,0,002,001


In [209]:
# Remove empty strings

def replace_empty_values(df):
    df_cl = df.replace(r'\s+', np.nan, regex=True)
    df_cl = df.replace('', np.nan)
    return df_cl

df = replace_empty_values(df)

In [210]:
df.head()

,COUNTY_ID,PIN,BLDG_NUM,PREFIX_DIR,PREFIXTYPE,STREETNAME,STREETTYPE,SUFFIX_DIR,UNIT_INFO,CITY,...,SCHOOL_DST,WSHD_DIST,GREEN_ACRE,OPEN_SPACE,AG_PRESERV,AGPRE_ENRD,AGPRE_EXPD,PARC_CODE,Shape_area,Shape_len
0,053,0911821410021,54,NaN,NaN,ADDRESS UNASSIGNED,NaN,NaN,NaN,CRYSTAL,...,281,Shingle Creek,N,N,N,NaN,NaN,0,002,001
1,053,1311821120005,4400 1/2,NaN,NaN,LYNDALE AVE N,NaN,NaN,NaN,MINNEAPOLIS,...,001,Shingle Creek,N,N,N,NaN,NaN,0,002,001
2,053,1311724440113,2325,NaN,NaN,MONTCLAIR LA,NaN,NaN,NaN,MOUND,...,NaN,NaN,N,N,N,NaN,NaN,0,002,001
3,053,2911821440091,28,NaN,NaN,ADDRESS UNASSIGNED,NaN,NaN,NaN,GOLDEN VALLEY,...,281,Bassett Creek,N,N,N,NaN,NaN,0,002,001
4,053,0111823440003,80,NaN,NaN,ADDRESS UNASSIGNED,NaN,NaN,NaN,MEDINA,...,284,NaN,N,N,N,NaN,NaN,0,002,001


In [211]:
# convert sale date to date object
import time

def convert_date(df):
    convert = lambda x : time.strptime(x, "%Y%m%d") if type(x) == str else np.nan
    df['sale_date_object'].apply(convert)
    
df['sale_date_object'] = df['SALE_DATE']

convert_date(df)

In [212]:
df['SALE_VALUE'].replace('[\$,)]','', inplace=True, regex=True)
df['SALE_VALUE'] = df['SALE_VALUE'].astype(float, copy=False)

df['YEAR_BUILT'] = df['YEAR_BUILT'].astype(int, copy=False)

In [213]:
df['FIN_SQ_FT'] = df['FIN_SQ_FT'].astype(int, copy=False)
df['EMV_LAND'] = df['EMV_LAND'].astype(float, copy=False)
df['EMV_BLDG'] = df['EMV_BLDG'].astype(float, copy=False)
df['EMV_TOTAL'] = df['EMV_TOTAL'].astype(float, copy=False)
df['TOTAL_TAX'] = df['TOTAL_TAX'].astype(float, copy=False)
df['TAX_CAPAC'] = df['TAX_CAPAC'].astype(float, copy=False)
df['SCHOOL_DST'] = df['SCHOOL_DST'].astype(float, copy=False)

In [214]:
df['cost_per_sqft'] = df['SALE_VALUE'] / df['FIN_SQ_FT']

In [215]:
df.head()

,COUNTY_ID,PIN,BLDG_NUM,PREFIX_DIR,PREFIXTYPE,STREETNAME,STREETTYPE,SUFFIX_DIR,UNIT_INFO,CITY,...,GREEN_ACRE,OPEN_SPACE,AG_PRESERV,AGPRE_ENRD,AGPRE_EXPD,PARC_CODE,Shape_area,Shape_len,sale_date_object,cost_per_sqft
0,053,0911821410021,54,NaN,NaN,ADDRESS UNASSIGNED,NaN,NaN,NaN,CRYSTAL,...,N,N,N,NaN,NaN,0,002,001,NaN,NaN
1,053,1311821120005,4400 1/2,NaN,NaN,LYNDALE AVE N,NaN,NaN,NaN,MINNEAPOLIS,...,N,N,N,NaN,NaN,0,002,001,NaN,NaN
2,053,1311724440113,2325,NaN,NaN,MONTCLAIR LA,NaN,NaN,NaN,MOUND,...,N,N,N,NaN,NaN,0,002,001,NaN,NaN
3,053,2911821440091,28,NaN,NaN,ADDRESS UNASSIGNED,NaN,NaN,NaN,GOLDEN VALLEY,...,N,N,N,NaN,NaN,0,002,001,NaN,NaN
4,053,0111823440003,80,NaN,NaN,ADDRESS UNASSIGNED,NaN,NaN,NaN,MEDINA,...,N,N,N,NaN,NaN,0,002,001,NaN,NaN


In [216]:
df_mpls = df[df['CITY'] == "MINNEAPOLIS"]

In [217]:
len(df_mpls)

129719

In [224]:
# Extracted only the columns I want to use for this analysis

sub_cols = ['PIN', 'USE1_DESC','EMV_TOTAL', 'TAX_CAPAC', 'TOTAL_TAX', 'FIN_SQ_FT', 'YEAR_BUILT']
df_subset = df_mpls[sub_cols]

In [225]:
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129719 entries, 1 to 429557
Data columns (total 7 columns):
PIN           129719 non-null object
USE1_DESC     129718 non-null object
EMV_TOTAL     129719 non-null float64
TAX_CAPAC     129719 non-null float64
TOTAL_TAX     129719 non-null float64
FIN_SQ_FT     129719 non-null int64
YEAR_BUILT    129719 non-null int64
dtypes: float64(3), int64(2), object(2)
memory usage: 7.9+ MB


In [226]:
df_subset.head()

,PIN,USE1_DESC,EMV_TOTAL,TAX_CAPAC,TOTAL_TAX,FIN_SQ_FT,YEAR_BUILT
1,1311821120005,Commercial,0.0,0.0,0.0,0,0
12,3502924230001,Commercial,0.0,0.0,0.0,0,0
13,0202824230266,NaN,0.0,0.0,0.0,0,0
14,0902924420203,Residential,500.0,6.0,9.0,0,0
15,0902924420198,Residential,61500.0,615.0,978.0,0,2001


In [227]:
with open('../data/processed/0202_mpls_parcel_data_abr.pkl', 'wb') as picklefile:
    pickle.dump(df_subset, picklefile)